In [4]:
from utils.table_mapping import *
import pandas as pd
import dash
from dash import dash_table, dcc, html, Input, Output
import numpy as np

# Generate random data for testing
def generate_random_data():
    np.random.seed(0)
    dates = pd.date_range(start="2020-01-01", periods=100)
    data1 = np.random.rand(100)
    data2 = np.random.rand(100)
    
    df1 = pd.DataFrame({
        'period': dates,
        'name': 'random_data1',
        'value': data1
    })

    df2 = pd.DataFrame({
        'period': dates,
        'name': 'random_data2',
        'value': data2
    })
    
    combined_df = pd.concat([df1, df2], ignore_index=True)
    combined_df.to_csv('./data/wps_gte_2015_pivot.csv', index=False)

generate_random_data()

# Function to generate the main DataFrame
def generate_main_table():
    df = pd.read_csv('./data/wps_gte_2015_pivot.csv', parse_dates=['period'])
    df['period'] = df['period'].dt.strftime('%m/%d')
    df = df.tail(2)
    df = df.set_index('period')
    df = df.T.reset_index()
    df['change'] = df.iloc[:, 2] - df.iloc[:, 1]
    df = df.rename(columns={'index': 'name'})
    df.columns.name = None
    return df

# Function to generate the DataTable
def generate_dash_table(df, idents, table_id, table_width):
    df = df.copy()
    idents_ids = list(idents.keys())
    idents_names = list(idents.values())
    df = df[df['name'].isin(idents_ids)]
    df = df.set_index('name')
    df = df.reindex(idents_ids)
    df = df.reset_index()
    df['name'] = df['name'].replace(idents_ids, idents_names)
    df = df.rename(columns={'name': table_id})
    df = df.reset_index(drop=True)
    return html.Div(
        dash_table.DataTable(
            id=table_id,
            columns=[{"name": i, "id": i, "type": "text", "presentation": "markdown"} for i in df.columns],
            data=df.to_dict('records'),
            tooltip_data=[
                {
                    column: {'value': str(value), 'type': 'text'}
                    for column, value in row.items()
                } for row in df.to_dict('records')
            ],
            tooltip_duration=None,
            style_table={'width': table_width}
        ),
        style={'margin-bottom': '20px'}
    )

# Function to generate historical graph
def generate_historical_graph(name):
    df = pd.read_csv('./data/wps_gte_2015_pivot.csv', parse_dates=['period'])
    df = df[df['name'] == name]
    fig = {
        'data': [{'x': df['period'], 'y': df['value'], 'type': 'line', 'name': name}],
        'layout': {'title': f'Historical Data for {name}'}
    }
    return dcc.Graph(figure=fig)

# Dash App Layout
app = dash.Dash(__name__)
app.layout = html.Div([
    html.Div(id='main-table-div'),
    html.Div(id='hovered-graph-div')
])

# Initializing the DataFrame and identifiers
idents = {
    'random_data1': 'Random Data 1',
    'random_data2': 'Random Data 2'
}

# Callbacks
@app.callback(
    Output('main-table-div', 'children'),
    Input('main-table-div', 'id')
)
def update_main_table(_):
    df = generate_main_table()
    return generate_dash_table(df, idents, 'main-table', '100%')

@app.callback(
    Output('hovered-graph-div', 'children'),
    Input('main-table', 'active_cell'),
    Input('main-table', 'data')
)
def update_graph_on_hover(active_cell, table_data):
    if active_cell:
        name = table_data[active_cell['row']]['name']
        return generate_historical_graph(name)
    return html.Div("Hover over a cell to see the historical graph.")

if __name__ == '__main__':
    app.run_server(debug=True, port=8053)
